<a href="https://colab.research.google.com/github/veiro/ADS2020/blob/master/imputacion/5_MIDA_con_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEER
1. Cambiar en enterno a GPU y la variable cuda= True
2. Para ejecutar todo el dataset, MODO_DESARROLLO=False, USAR_GOOGLE_DRIVE =  False y ejecutar un docker local de colab por que demora mas de 12 hrs.

## 1. Requirements

In [1]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [17]:
import os
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [3]:
if (USAR_GOOGLE_DRIVE):
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_IMPUTATION = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/imputacion/"
else:
  PATH_DATA_PROCESADA = "/mnt"
  PATH_IMPUTATION = "/mnt/"


Mounted at /content/gdrive


## 2. Set Args

In [4]:
theta = 7
if (MODO_DESARROLLO):
  num_epochs = 50 #500 con este valor demora mucho
else:
  num_epochs = 500
dropout_ratio = 0.5

#data_path = '/BostonHousing.csv'
mechanism = 'mcar'
method = 'uniform'

test_size = 0.3
use_cuda = True
batch_size  = 128 # not in the paper



## 3. Prepare Data

In [5]:
%%time

#data = pd.read_csv(PATH_DATA_PROCESADA+data_path).values
data_filepath = PATH_DATA_PROCESADA+'/csse_covid_19_data.csv'
data_full = pd.read_csv(data_filepath)
data_sin_columnas_completas = data_full.drop(['FIPS', 'Admin2', 'Recovered', 'Active', 'Combined_Key'], axis=1)
data_sin_null = data_sin_columnas_completas.dropna(subset=['Province_State', 'Lat', 'Long_', 'Incident_Rate', 'Case_Fatality_Ratio'])

if(MODO_DESARROLLO):
   data_utilizar = data_sin_null.sample(n=10000, random_state=1)
else:
  data_utilizar = data_sin_null

CPU times: user 6.67 s, sys: 1.08 s, total: 7.75 s
Wall time: 15.3 s


In [6]:
# variables gloables
encoded_df_Country_Region_cols = 0
encoderOHE = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoderLE1 = LabelEncoder()
encoderLE2 = LabelEncoder()
encoderLE3 = LabelEncoder()



# ENCODING
# 1- onehotenconding de Country_Region
df=data_utilizar

encoded_data = encoderOHE.fit_transform( df[['Country_Region']])
encoded_df_Country_Region = pd.DataFrame(encoded_data, columns=encoderOHE.get_feature_names_out(['Country_Region']))
encoded_df_Country_Region_cols = encoded_df_Country_Region.shape[1]

df = df.drop(['Country_Region'], axis=1)

# hice este concat horrible por que por alguna razon no me andaba bien el concant ordinario
pos = encoded_df_Country_Region_cols
for col in df.columns:
  encoded_df_Country_Region.insert( pos, col, df[col].to_numpy(), True)
  pos=pos+1

df = encoded_df_Country_Region

# 2 Province_State, Last_Update, date
df['Province_State_encoded'] = encoderLE1.fit_transform(df['Province_State'])
df = df.drop(['Province_State'], axis=1)

df['Last_Update_encoded'] = encoderLE2.fit_transform(df['Last_Update'])
df = df.drop(['Last_Update'], axis=1)

df['date_encoded'] = encoderLE3.fit_transform(df['date'])
df = df.drop(['date'], axis=1)
  #return df,  encoded_df_Country_Region_cols

data=df

data = data.reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
rows, cols = data.shape
#shuffled_index = np.random.permutation(rows)
#train_index = shuffled_index[:int(rows*(1-test_size))]
#test_index = shuffled_index[int(rows*(1-test_size)):]
#train_data = data[train_index, :]
#test_data = data[test_index, :]

train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)


# standardized between 0 and 1
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [8]:
def missing_method(raw_data, mechanism='mcar', method='uniform') :

    data = raw_data.copy()
    rows, cols = data.shape

    # missingness threshold
    t = 0.2

    if mechanism == 'mcar' :

        if method == 'uniform' :
            # uniform random vector
            v = np.random.uniform(0., 1.,size=(rows, cols))

            # missing values where v<=t
            mask = (v<=t)
            data[mask] = 0

        elif method == 'random' :
            # only half of the attributes to have missing value
            missing_cols = np.random.choice(cols, cols//2)
            c = np.zeros(cols, dtype=bool)
            c[missing_cols] = True

            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where v<=t
            mask = (v<=t)*c
            data[mask] = 0

        else :
            print("Error : There are no such method")
            raise

    elif mechanism == 'mnar' :

        if method == 'uniform' :
            # randomly sample two attributes
            sample_cols = np.random.choice(cols, 2)

            # calculate ther median m1, m2
            m1, m2 = np.median(data[:,sample_cols], axis=0)
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = data[:,sample_cols[0]] <= m1
            m2 = data[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]

            mask = m*(v<=t)
            data[mask] = 0


        elif method == 'random' :
            # only half of the attributes to have missing value
            missing_cols = np.random.choice(cols, cols//2)
            c = np.zeros(cols, dtype=bool)
            c[missing_cols] = True

            # randomly sample two attributes
            sample_cols = np.random.choice(cols, 2)

            # calculate ther median m1, m2
            m1, m2 = np.median(data[:,sample_cols], axis=0)
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = data[:,sample_cols[0]] <= m1
            m2 = data[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]

            mask = m*(v<=t)*c
            data[mask] = 0

        else :
            print("Error : There is no such method")
            raise

    else :
        print("Error : There is no such mechanism")
        raise

    return data, mask

In [9]:
missed_data, mask = missing_method(test_data, mechanism=mechanism, method=method)

missed_data = torch.from_numpy(missed_data).float()
train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

## 4. Define Model

In [10]:
device = torch.device("cuda" if use_cuda else "cpu")

In [11]:
class Autoencoder(nn.Module):
    def __init__(self, dim):
        super(Autoencoder, self).__init__()
        self.dim = dim

        self.drop_out = nn.Dropout(p=0.5)

        self.encoder = nn.Sequential(
            nn.Linear(dim+theta*0, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*3)
        )

        self.decoder = nn.Sequential(
            nn.Linear(dim+theta*3, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*0)
        )

    def forward(self, x):
        x = x.view(-1, self.dim)
        x_missed = self.drop_out(x)

        z = self.encoder(x_missed)
        out = self.decoder(z)

        out = out.view(-1, self.dim)

        return out

In [12]:
model = Autoencoder(dim=cols).to(device)

## 5. Define Loss and Optimizer

In [13]:
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), momentum=0.99, lr=0.01, nesterov=True)

## 6. Train Model

In [ ]:
%%time
cost_list = []
early_stop = False

for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size

    for i, batch_data in enumerate(train_loader):

        batch_data = batch_data.to(device)

        reconst_data = model(batch_data)
        cost = loss(reconst_data, batch_data)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % (total_batch//2) == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.6f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

        # early stopping rule 1 : MSE < 1e-06
        if cost.item() < 1e-06 :
            early_stop = True
            break

#         early stopping rule 2 : simple moving average of length 5
#         sometimes it doesn't work well.
#         if len(cost_list) > 5 :
#            if cost.item() > np.mean(cost_list[-5:]):
#                early_stop = True
#                break

        cost_list.append(cost.item())

    if early_stop :
        break

print("Learning Finished!")

Epoch [1/500], lter [8181/16363], Loss: 0.008732
Epoch [1/500], lter [16362/16363], Loss: 0.010721
Epoch [2/500], lter [8181/16363], Loss: 0.009175
Epoch [2/500], lter [16362/16363], Loss: 0.008951
Epoch [3/500], lter [8181/16363], Loss: 0.006381
Epoch [3/500], lter [16362/16363], Loss: 0.008357
Epoch [4/500], lter [8181/16363], Loss: 0.007271
Epoch [4/500], lter [16362/16363], Loss: 0.006288
Epoch [5/500], lter [8181/16363], Loss: 0.006275
Epoch [5/500], lter [16362/16363], Loss: 0.007862
Epoch [6/500], lter [8181/16363], Loss: 0.006448
Epoch [6/500], lter [16362/16363], Loss: 0.006123
Epoch [7/500], lter [8181/16363], Loss: 0.006429
Epoch [7/500], lter [16362/16363], Loss: 0.006083
Epoch [8/500], lter [8181/16363], Loss: 0.004572
Epoch [8/500], lter [16362/16363], Loss: 0.006617
Epoch [9/500], lter [8181/16363], Loss: 0.006565
Epoch [9/500], lter [16362/16363], Loss: 0.006776
Epoch [10/500], lter [8181/16363], Loss: 0.006159
Epoch [10/500], lter [16362/16363], Loss: 0.006828
Epoch [1

## 7. Test Model

In [ ]:
model.eval()
filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = 0

for i in range(cols) :
    if mask[:,i].sum() > 0 :
        y_actual = test_data[:,i][mask[:,i]]
        y_predicted = filled_data[:,i][mask[:,i]]

        rmse = sqrt(mean_squared_error(y_actual, y_predicted))
        rmse_sum += rmse

print("RMSE_SUM :", rmse_sum)
df = pd.DataFrame(filled_data, columns = data.columns)
df.head(5)
#df.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_MIDA_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)


RMSE_SUM : 5.9289848676449


,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,0.008653,0.014528,0.036974,0.041412,0.028605,0.060135,0.122504,-0.000576,0.036309,0.029245,...,0.049589,0.586695,0.519050,0.016261,0.031331,0.030735,0.022137,0.206066,0.520436,0.094763
1,0.001135,-0.000399,0.003415,0.040890,0.002089,-0.004704,-0.000890,-0.000889,0.001007,0.001751,...,-0.000157,0.773647,0.150486,0.002407,0.004120,0.031559,0.013717,0.446388,0.251358,0.588798
2,0.002926,0.004183,0.009825,0.029773,0.006070,0.027643,0.147322,-0.009506,0.019506,0.001569,...,0.023961,0.581849,0.378993,0.015010,0.037723,-0.005775,0.033490,0.280515,0.147346,0.326824
3,0.006853,0.003655,-0.016297,0.409209,0.002442,0.011207,-0.015221,-0.008741,0.002156,-0.001270,...,0.010374,0.823619,0.134146,0.000402,-0.001149,0.032177,0.009938,0.544385,0.336562,0.039943
4,0.000359,-0.000261,0.002396,-0.001090,0.002605,0.001881,0.004770,0.000741,-0.002743,0.003443,...,-0.000295,0.722401,0.271837,0.000028,0.000679,0.023244,0.014731,0.497917,0.128828,0.321995


## Guardo el modelo

In [ ]:
torch.save(model, PATH_DATA_PROCESADA+'/model_csse_covid_19_data_MIDA_Desarrollo='+str(MODO_DESARROLLO))

In [13]:
model = torch.load(PATH_DATA_PROCESADA+'/model_csse_covid_19_data_MIDA_Desarrollo='+str(MODO_DESARROLLO))
model.eval()

Autoencoder(
  (drop_out): Dropout(p=0.5, inplace=False)
  (encoder): Sequential(
    (0): Linear(in_features=34, out_features=41, bias=True)
    (1): Tanh()
    (2): Linear(in_features=41, out_features=48, bias=True)
    (3): Tanh()
    (4): Linear(in_features=48, out_features=55, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=55, out_features=48, bias=True)
    (1): Tanh()
    (2): Linear(in_features=48, out_features=41, bias=True)
    (3): Tanh()
    (4): Linear(in_features=41, out_features=34, bias=True)
  )
)

# Generar dataset-mida



In [14]:
# standardized between 0 and 1
scaler = MinMaxScaler()
scaler.fit(data)
data_scaled = scaler.transform(data)

missed_data_full, mask_full = missing_method(data_scaled, mechanism=mechanism, method=method)

missed_data_full = torch.from_numpy(missed_data_full).float()

model.eval()
filled_data_full = model(missed_data_full.to(device))
filled_data_full = filled_data_full.cpu().detach().numpy()

rmse_sum = 0

for i in range(cols) :
    if missed_data_full[:,i].sum() > 0 :
        y_actual = data_scaled[:,i][mask_full[:,i]]
        y_predicted = filled_data_full[:,i][mask_full[:,i]]

        rmse = sqrt(mean_squared_error(y_actual, y_predicted))
        rmse_sum += rmse

print("RMSE_SUM  total:", rmse_sum)


filled_data_full_unscaled = scaler.inverse_transform(filled_data_full)


RMSE_SUM  total: 6.0820214508632535


In [15]:
df = pd.DataFrame(filled_data_full_unscaled, columns = data.columns)
df.head(5)

,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,0.009431,-0.005196,0.121716,0.031616,0.083824,-0.002819,-0.319216,-0.280827,0.327357,-0.024405,...,0.130371,-25.362644,38.497833,3.681294e+05,7078.833496,47844.535156,1.172194,151.796738,363.079468,392.138702
1,0.012787,-0.014851,0.135592,0.085301,0.005315,-0.003589,-0.255199,-0.374507,0.483833,-0.018800,...,-0.093106,-35.848209,74.772392,3.250693e+05,8641.758789,8772.288086,1.572035,160.581772,349.444977,388.419983
2,0.016859,-0.042499,0.095538,0.077159,0.012499,-0.000261,-0.303056,-0.445762,0.464097,-0.061002,...,-0.026218,-34.560398,79.142586,3.458896e+05,8439.585938,18890.322266,1.522529,210.251297,362.835297,385.251587
3,-0.000120,-0.006204,0.650915,-0.036084,0.051961,-0.056608,-0.067763,0.006102,0.020401,-0.019808,...,0.703257,-0.014398,-77.229294,1.586463e+06,27012.812500,58894.054688,1.190427,164.836807,109.396774,168.578705
4,0.012600,-0.035848,0.079237,0.079591,-0.000363,-0.010666,-0.224636,-0.385734,0.542262,-0.020466,...,-0.035439,-37.401093,70.354286,4.094745e+05,9571.671875,7719.047363,1.529568,182.970428,342.590332,385.884827


In [17]:
df.shape

(2992223, 34)

In [18]:
%%time
# DECODE
# 1- onehotenconding de Country_Region.
encoded_df_Country_Region = df.iloc[:,  :encoded_df_Country_Region_cols]
df['Country_Region'] = encoderOHE.inverse_transform(encoded_df_Country_Region)
df.drop(columns=df.columns[:encoded_df_Country_Region_cols], axis=1,  inplace=True)



CPU times: user 488 ms, sys: 50.9 ms, total: 539 ms
Wall time: 519 ms


In [19]:

#2- Province_State, Last_Update, date

Province_State_encoded_count = data.Province_State_encoded.unique().tolist().count(1)
Last_Update_encoded_count = data.Last_Update_encoded.unique().tolist().count(1)
date_encoded_encoded_count = data.date_encoded.unique().tolist().count(1)


df['Province_State_encoded'] = df['Province_State_encoded'].astype(int)
df['date_encoded'] = df['date_encoded'].astype(int)
df['Last_Update_encoded'] = df['Last_Update_encoded'].astype(int)

for i,  row in tqdm(df.iterrows()):
  # en algunos casos inventa valores que no existen, entonces como es un error le asigno el primero pero puede ser cualquier valor.

  if(row['Province_State_encoded'] >Province_State_encoded_count or row['Province_State_encoded'] <1 ):
    #significa que genero algo que no existe
    df.at[i,'Province_State_encoded'] = 1


  if(row['Last_Update_encoded'] <1 or row['Last_Update_encoded'] > Last_Update_encoded_count ):
    #significa que genero algo que no existe
    df.at[i,'Last_Update_encoded'] = 1


  if(row['date_encoded'] <1 or row['date_encoded'] > date_encoded_encoded_count):
    #significa que genero algo que no existe
    df.at[i,'date_encoded'] = 1




2992223it [05:40, 8797.90it/s]


In [20]:

df['Province_State'] = encoderLE1.inverse_transform(df['Province_State_encoded'].astype(int) )
df['Last_Update'] = encoderLE2.inverse_transform(df['Last_Update_encoded'].astype(int) )
df['date'] = encoderLE3.inverse_transform(df['date_encoded'].astype(int) )



df = df.drop(['Province_State_encoded'], axis=1)
df = df.drop(['Last_Update_encoded'], axis=1)
df = df.drop(['date_encoded'], axis=1)

#3- ordeno columnas

col = df.pop('Province_State')
df.insert(0, col.name, col)

col = df.pop('Country_Region')
df.insert(1,  col.name, col)

col = df.pop('Last_Update')
df.insert(2,  col.name, col)

# cambio tipo de dato

df['Confirmed'] = df['Confirmed'].astype(int)

df['Deaths'] = df['Deaths'].astype(int)


# cantidad de cifras significativas>

df["Lat"] = round(df['Lat'],6)
df["Long_"] = round(df['Long_'],4)


In [21]:
df.shape

(2992223, 10)

In [22]:
df.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,date
0,Acre,Spain,2021-01-02 05:22:33,-25.362644,38.497799,368129,7078,47844.535156,1.172194,01-01-2022
1,Acre,Pakistan,2021-01-02 05:22:33,-35.848209,74.772400,325069,8641,8772.288086,1.572035,01-01-2022
2,Acre,France,2021-01-02 05:22:33,-34.560398,79.142601,345889,8439,18890.322266,1.522529,01-01-2022
3,Acre,United Kingdom,2021-01-02 05:22:33,-0.014398,-77.229301,1586463,27012,58894.054688,1.190427,01-01-2022
4,Acre,France,2021-01-02 05:22:33,-37.401093,70.354301,409474,9571,7719.047363,1.529568,01-01-2022


In [23]:
df.isna().sum()

Province_State         0
Country_Region         0
Last_Update            0
Lat                    0
Long_                  0
Confirmed              0
Deaths                 0
Incident_Rate          0
Case_Fatality_Ratio    0
date                   0
dtype: int64

In [24]:
# reseteo indices
data_utilizar = data_utilizar.reset_index(drop=True)
df = df.reset_index(drop=True)

diferrence = data_utilizar.compare(df)

In [25]:
diferrence

Province_State        Country_Region  \
                                                 self other            self   
0                        Australian Capital Territory  Acre       Australia   
1                                     New South Wales  Acre       Australia   
2                                  Northern Territory  Acre       Australia   
3                                          Queensland  Acre       Australia   
4                                     South Australia  Acre       Australia   
...                                               ...   ...             ...   
2992218                              Pitcairn Islands  Acre  United Kingdom   
2992219  Saint Helena, Ascension and Tristan da Cunha  Acre             NaN   
2992220                                      Scotland  Acre  United Kingdom   
2992221                      Turks and Caicos Islands  Acre  United Kingdom   
2992222                                         Wales  Acre  United Kingdom   

                                 Last_Update                           Lat  \
                  other                 self                other     self   
0                 Spain                  NaN                  NaN -35.4735   
1              Pakistan                  NaN                  NaN -33.8688   
2                France                  NaN                  NaN -12.4634   
3        United Kingdom                  NaN                  NaN -27.4698   
4                France                  NaN                  NaN -34.9285   
...                 ...                  ...                  ...      ...   
2992218       Australia  2023-03-10 04:21:03  2021-01-02 05:22:33 -24.3768   
2992219             NaN  2023-03-10 04:21:03  2021-01-02 05:22:33  -7.9467   
2992220          Russia  2023-03-10 04:21:03  2021-01-02 05:22:33  56.4907   
2992221           China  2023-03-10 04:21:03  2021-01-02 05:22:33  21.6940   
2992222          France  2023-03-10 04:21:03  2021-01-02 05:22:33  52.1307   

                       Long_             Confirmed              Deaths  \
             other      self      other       self      other     self   
0       -25.362644  149.0124  38.497799      118.0   368129.0      3.0   
1       -35.848209  151.2093  74.772400     4947.0   325069.0     54.0   
2       -34.560398  130.8456  79.142601       75.0   345889.0      0.0   
3        -0.014398  153.0251 -77.229301     1255.0  1586463.0      6.0   
4       -37.401093  138.6007  70.354301      580.0   409474.0      4.0   
...            ...       ...        ...        ...        ...      ...   
2992218 -51.802525 -128.3242  78.633797        4.0   540063.0      0.0   
2992219 -16.116199  -14.3559 -95.468201     2166.0   144749.0      0.0   
2992220  20.027695   -4.2026   7.614000  2157720.0   381519.0  16806.0   
2992221 -25.712908  -71.7979 -44.645699     6561.0   325692.0     38.0   
2992222  -7.096838   -3.7837 -23.182600   898118.0   107609.0  11500.0   

                 Incident_Rate                Case_Fatality_Ratio            \
           other          self          other                self     other   
0         7078.0     27.563653   47844.535156            2.542373  1.172194   
1         8641.0     60.938655    8772.288086            1.091571  1.572035   
2         8439.0     30.537459   18890.322266            0.000000  1.522529   
3        27012.0     24.533281   58894.054688            0.478088  1.190427   
4         9571.0     33.020211    7719.047363            0.689655  1.529568   
...          ...           ...            ...                 ...       ...   
2992218   3021.0   5970.149254   80778.007812            0.000000  0.220583   
2992219   -686.0  38261.791203  134923.265625            0.000000  0.351054   
2992220   4925.0  39494.810829   81632.304688            0.778878  1.653584   
2992221   6395.0  16945.606695  -27098.083984            0.579180  1.840343   
2992222   2333.0  28615.242465  117832.273438            1.280455  0.922066   

     

In [26]:
df.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_MIDA_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

7 hrs de ejecucion en modo MODO_DESARROLLO = false